In [5]:
import sys
sys.path.append('/diffusers/src')

from diffusers.models.attention_processor import (
    Attention,
    AttnProcessor,
    AttentionProcessor,
    FluxAttnProcessor2_0,
    FluxAttnProcessor2_0_NPU,
    FusedFluxAttnProcessor2_0,
)

In [ ]:
import torch
from diffusers import AutoPipelineForText2Image
from safetensors.torch import load_file

# 加载基础模型管道
flux_pipe = AutoPipelineForText2Image.from_pretrained("/mnt/d/studying-code/modelscope/FLUX.1-dev", torch_dtype=torch.bfloat16)
flux_pipe.enable_sequential_cpu_offload()

In [ ]:
prompt = ["A photo of a bunny"]

image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    height=1024,
    width=1024,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]

In [ ]:
for k in flux_pipe.transformer.attn_processors.keys():
    print(k)

In [6]:
class EmbeddingInsertAttnProcessor(AttnProcessor):
    def __init__(self):
        super().__init__()
        self.first_maps = {}
        self.prev_maps = {}

    def __call__(self, attn: Attention, hidden_states, encoder_hidden_states=None, **kwargs):
        if encoder_hidden_states is None:
            # Is self attention
            res = super().__call__(attn, hidden_states, encoder_hidden_states, **kwargs)
            cross_map = torch.cat((self.first_maps[t], self.prev_maps[t]), dim=1)
            res = super().__call__(attn, hidden_states, cross_map, **kwargs)
    
        else:
            # Is cross attention
            cross_map = torch.cat((self.first_maps[t], self.prev_maps[t]), dim=1)
            res = super().__call__(attn, hidden_states, cross_map, **kwargs)
    
        return res

In [14]:
for index_block, block in enumerate(flux_pipe.transformer.transformer_blocks):
    encoder_hidden_states, hidden_states = block(
        hidden_states=hidden_states,
        encoder_hidden_states=encoder_hidden_states,
        temb=temb,
        image_rotary_emb=image_rotary_emb,
        joint_attention_kwargs=joint_attention_kwargs,
    )

0 FluxTransformerBlock(
  (norm1): AdaLayerNormZero(
    (silu): SiLU()
    (linear): Linear(in_features=3072, out_features=18432, bias=True)
    (norm): LayerNorm((3072,), eps=1e-06, elementwise_affine=False)
  )
  (norm1_context): AdaLayerNormZero(
    (silu): SiLU()
    (linear): Linear(in_features=3072, out_features=18432, bias=True)
    (norm): LayerNorm((3072,), eps=1e-06, elementwise_affine=False)
  )
  (attn): Attention(
    (norm_q): RMSNorm()
    (norm_k): RMSNorm()
    (to_q): Linear(in_features=3072, out_features=3072, bias=True)
    (to_k): Linear(in_features=3072, out_features=3072, bias=True)
    (to_v): Linear(in_features=3072, out_features=3072, bias=True)
    (add_k_proj): Linear(in_features=3072, out_features=3072, bias=True)
    (add_v_proj): Linear(in_features=3072, out_features=3072, bias=True)
    (add_q_proj): Linear(in_features=3072, out_features=3072, bias=True)
    (to_out): ModuleList(
      (0): Linear(in_features=3072, out_features=3072, bias=True)
      (